<a href="https://colab.research.google.com/github/rdkworld/AIPND-2022/blob/main/Generalized/Train_an_Existing_Pytorch_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Pre-requisite Setup

###Get Libraries

In [1]:
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

torch version: 1.12.1+cu113
torchvision version: 0.13.1+cu113


###Regular Imports

In [11]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

#Additions from functions
import os
import tarfile

###Helpers/functions from Github

In [110]:
# Try to import the helper functions, download it from GitHub if it doesn't work
try:
    import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves
except:
    # Get the scripts
    print("[INFO] Couldn't find the scripts... downloading them from GitHub.")
    !git clone https://github.com/rdkworld/AIPND-2022
    !mv AIPND-2022/Generalized/*.py .
    !rm -rf AIPND-2022
    import data_setup, engine, model_builder, utils 
    from helper_functions import download_data, set_seeds, plot_loss_curves

In [142]:
print("[INFO] Couldn't find the scripts... downloading them from GitHub.")
!git clone https://github.com/rdkworld/AIPND-2022
!mv AIPND-2022/Generalized/*.py .
!rm -rf AIPND-2022
import data_setup, engine, model_builder, utils 
from helper_functions import download_data, set_seeds, plot_loss_curves

[INFO] Couldn't find the scripts... downloading them from GitHub.
Cloning into 'AIPND-2022'...
remote: Enumerating objects: 304, done.
remote: Counting objects: 100% (194/194), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 304 (delta 98), reused 116 (delta 53), pack-reused 110
Receiving objects: 100% (304/304), 11.00 MiB | 41.27 MiB/s, done.
Resolving deltas: 100% (120/120), done.


### Connect Colab and Google Drive to save and load models

In [4]:
#Mount Google Drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Setup target device

In [5]:
# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## User Input Parameters including Hyperparameters

In [79]:
#Data
SOURCE_URL = 'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz'
BASE_DATA_DIRECTORY = 'data'
PROJECT_DATA_DIRECTORY = 'flowers'
FILE_NAME = 'flowers.tar.gz'
train_dir = f"{BASE_DATA_DIRECTORY}/{PROJECT_DATA_DIRECTORY}/train"
valid_dir = f"{BASE_DATA_DIRECTORY}/{PROJECT_DATA_DIRECTORY}/valid"
test_dir = f"{BASE_DATA_DIRECTORY}/{PROJECT_DATA_DIRECTORY}/test"

# Setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 64
HIDDEN_UNITS = '' #Not used
LEARNING_RATE = 0.003
MODEL_NAME = 'vit_b_16'
MODEL_WEIGHT = 'ViT_B_16' 
LOSS_FUNCTION = 'CrossEntropyLoss'
OPTIMIZER = 'Adam'
MANUAL_RESIZE = 64
NUM_CLASSES = 102
FEATURE_EXTRACT = True
RGB = 3 #(Color picture is 3, black & white is 1) 


##Download data and categorize into train/valid/test folders as required

In [65]:
#Work in Progress, See next cell
!ls $BASE_DATA_DIRECTORY/
!ls

flowers
'$BASE_DATA_DIRECTORY'	 drive		       model_builder.py   sample_data
 data			 engine.py	       predictions.py	  train.py
 data_setup.py		 helper_functions.py   __pycache__	  utils.py


In [9]:
#Temporarily copy from Google Drive
!mkdir $BASE_DATA_DIRECTORY
!cp /content/drive/MyDrive/flowers.tar.gz data/ #copy from drive to colab

In [31]:
#Untar the file
with tarfile.open(os.path.join(BASE_DATA_DIRECTORY, FILE_NAME), "r") as tar_ref:
    print(f"[INFO] Unzipping {FILE_NAME}...") 
    tar_ref.extractall(BASE_DATA_DIRECTORY)

FileNotFoundError: ignored

In [ ]:
#!tar xf $BASE_DATA_DIRECTORY/$FILE_NAME

In [28]:
if os.path.exists(os.path.join(BASE_DATA_DIRECTORY, PROJECT_DATA_DIRECTORY)):
  os.remove(os.path.join(BASE_DATA_DIRECTORY, FILE_NAME))

## Get info on Pre-Trained Models

###Pre-trained Model & Transform Details

In [91]:
#Get pre-trained model weights and model
pretrained_weights = eval(f"torchvision.models.{MODEL_WEIGHT}_Weights.DEFAULT")
pretrained_model = eval(f"torchvision.models.{MODEL_NAME}(weights = pretrained_weights)").to(device)
auto_transforms = pretrained_weights.transforms()

In [94]:
# Print a summary using torchinfo (uncomment for actual output)
summary(model=pretrained_model,
        input_size= (BATCH_SIZE, RGB, auto_transforms.crop_size[0], auto_transforms.crop_size[0]),  # make sure this is "input_size", not "input_shape"
        col_names=["input_size", "output_size", "num_params", "trainable"],  # col_names=["input_size"], # uncomment for smaller output
        col_width=20,
        row_settings=["var_names"]
), auto_transforms

(======================================================================================================================================================
 Layer (type (var_name))                                                Input Shape          Output Shape         Param #              Trainable
 VisionTransformer (VisionTransformer)                                  [64, 3, 224, 224]    [64, 1000]           768                  True
 ├─Conv2d (conv_proj)                                                   [64, 3, 224, 224]    [64, 768, 14, 14]    590,592              True
 ├─Encoder (encoder)                                                    [64, 197, 768]       [64, 197, 768]       151,296              True
 │    └─Dropout (dropout)                                               [64, 197, 768]       [64, 197, 768]       --                   --
 │    └─Sequential (layers)                                             [64, 197, 768]       [64, 197, 768]       --                   True
 │   

In [129]:
!cat model_builder.py

"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch
from torch import nn 
from torchvision import models
from torch import nn
from collections import OrderedDict

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def update_last_layer_pretrained_model(pretrained_model, num_classes, feature_extract):
    set_parameter_requires_grad(pretrained_model, feature_extract)
    if getattr(pretrained_model, 'heads'):
        num_ftrs = pretrained_model.heads.head.in_features
        pretrained_model.heads.head = nn.Linear(num_ftrs, num_classes, bias = True)
    return pretrained_model

def initialize_existing_models(model_name, model_type, num_classes, feature_extract, hidden_units, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these variables is model specific.
    model_ft = None
    input_si

In [146]:
from model_builder import update_last_layer_pretrained_model

ImportError: ignored

In [147]:
!ls

'$BASE_DATA_DIRECTORY'	 drive		       model_builder.py   sample_data
 data			 engine.py	       predictions.py	  train.py
 data_setup.py		 helper_functions.py   __pycache__	  utils.py


In [145]:
# Create model with help from model_builder.py
updated_pretrained_model = model_builder.update_last_layer_pretrained_model(pretrained_model, NUM_CLASSES, FEATURE_EXTRACT).to(device)
#updated_pretrained_model = update_last_layer_pretrained_model(pretrained_model, NUM_CLASSES, FEATURE_EXTRACT)

AttributeError: ignored

In [127]:
summary(model=updated_pretrained_model,
        input_size= (BATCH_SIZE, RGB, auto_transforms.crop_size[0], auto_transforms.crop_size[0]),  # make sure this is "input_size", not "input_shape"
        col_names=["input_size", "output_size", "num_params", "trainable"],  # col_names=["input_size"], # uncomment for smaller output
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                                Input Shape          Output Shape         Param #              Trainable
VisionTransformer (VisionTransformer)                                  [64, 3, 224, 224]    [64, 102]            768                  Partial
├─Conv2d (conv_proj)                                                   [64, 3, 224, 224]    [64, 768, 14, 14]    (590,592)            False
├─Encoder (encoder)                                                    [64, 197, 768]       [64, 197, 768]       151,296              False
│    └─Dropout (dropout)                                               [64, 197, 768]       [64, 197, 768]       --                   --
│    └─Sequential (layers)                                             [64, 197, 768]       [64, 197, 768]       --                   False
│    │    └─EncoderBlock (encoder_layer_0)                             [64, 197, 768]       [64, 197, 768]       (7,087,872)          False
│    │    └─Encod

In [109]:
from IPython.lib import pretty
if getattr(pretrained_model, 'heads'):
  print(True)
pretrained_model.heads.head

True


Linear(in_features=768, out_features=1000, bias=True)

###Dataloaders

In [95]:
# Create data loaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=auto_transforms,
    batch_size=BATCH_SIZE
)
if len(class_names) != NUM_CLASSES:
  print("Mismatch in the number of unique classes/labels and user input NUM_CLASSES")
  exit()

In [67]:
model_builder.update_last_layer_pretrained_model

('data/flowers/train', 'data/flowers/test')

In [125]:
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch
from torch import nn 
from torchvision import models
from torch import nn
from collections import OrderedDict

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def update_last_layer_pretrained_model(pretrained_model, num_classes, feature_extract):
    set_parameter_requires_grad(pretrained_model, feature_extract)
    if getattr(pretrained_model, 'heads'):
        num_ftrs = pretrained_model.heads.head.in_features
        pretrained_model.heads.head = nn.Linear(num_ftrs, num_classes, bias = True)
    return pretrained_model

def initialize_existing_models(model_name, model_type, num_classes, feature_extract, hidden_units, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet18":
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Sequential(
                        nn.Linear(num_ftrs, num_classes),
                        nn.LogSoftmax(dim=1))
        input_size = 224
    elif model_name == "alexnet":
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Sequential(
                                    nn.Linear(num_ftrs,num_classes),
                                    nn.LogSoftmax(dim=1))        
        input_size = 224
    elif model_name in ["vgg11_bn", "vgg13", "vgg16"]:
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Sequential(
                                    nn.Linear(num_ftrs,num_classes),
                                    nn.LogSoftmax(dim=1))
        input_size = 224
    elif model_name == "squeezenet":
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Sequential(
                                    nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1)),
                                    nn.LogSoftmax(dim=1))
        model_ft.num_classes = num_classes
        input_size = 224
    elif model_name == "densenet121":
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Sequential(
                                    nn.Linear(num_ftrs, num_classes),
                                    nn.LogSoftmax(dim=1))     
        input_size = 224
    elif model_name == "inception": # This model expects (299,299) sized images and has auxiliary output
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Sequential(
                                    nn.Linear(num_ftrs, num_classes),
                                    nn.LogSoftmax(dim=1))
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Sequential(
                            nn.Linear(num_ftrs,num_classes),
                            nn.LogSoftmax(dim=1))
        input_size = 299
    else:
        print("Invalid model name, please use one of the models supported by this application, exiting...")
        exit()
    return model_ft, input_size

#Get pre-trained model specifications and override with classifier portion with user activation units
def build_custom_models(model_name, model_type, num_classes, feature_extract, hidden_units, use_pretrained=True):
       
    model_ft = getattr(models, model_name)(pretrained = use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    if model_name == 'resnet18':
        in_features = model_ft.fc.in_features
    else:
        try: #Is there an iterable classifier layer for the model chosen?
            iter(model_ft.classifier)
        except TypeError: #If no, choose the classifier layer with no index
            in_features = model_ft.classifier.in_features
        else:
            try: #If yes, check if first index has in_features attribute
                in_features = model_ft.classifier[0].in_features
            except AttributeError: #If No, check if second index has in_features attribute
                in_features = model_ft.classifier[1].in_features
        
    hidden_layers = [in_features] + hidden_units
    layer_builder = (
        lambda i, v : (f"fc{i}", nn.Linear(hidden_layers[i-1], v)),
        lambda i, v: (f"relu{i}", nn.ReLU()),
        lambda i, v: (f"drop{i}", nn.Dropout())        
    )
    
    layers = [f(i, v) for i, v in enumerate(hidden_layers) if i > 0 for f in layer_builder]
    layers += [('fc_final', nn.Linear(hidden_layers[-1], num_classes)),
               ('output', nn.LogSoftmax(dim=1))]    

    if model_name == 'resnet18':
        fc = nn.Sequential(OrderedDict(layers))
        model_ft.fc = fc
    else:
        classifier = nn.Sequential(OrderedDict(layers))
        model_ft.classifier = classifier
#     print("AFTER")
#     print(model.classifier)
    
    return model_ft

#Define model/ neural network class
# class ImageClassifier(nn.Module):
#     def __init__(self):
#         super(ImageClassifer, self).__init__()
#         self.flatten = nn.Flatten()
#         self.model_stack = nn.Sequential(
#             nn.Linear(),
#             nn.ReLU(),
#             nn.Dropout(0.2),
#             nn.Linear(),
#             nn.LogSoftmax(dim=1)
#         )
               
#     def forward(self, x):
#         x = self.flatten(x)
#         logits = self.model_stack(x)
#         return logits
class TinyVGG(nn.Module):
    """Creates the TinyVGG architecture.

    Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
    See the original architecture here: https://poloclub.github.io/cnn-explainer/

    Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape, 
                    out_channels=hidden_units, 
                    kernel_size=3, 
                    stride=1, 
                    padding=0),  
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units, 
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2)
        )
        self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
          nn.Flatten(),
          # Where did this in_features shape come from? 
          # It's because each layer of our network compresses and changes the shape of our inputs data.
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x
        # return self.classifier(self.block_2(self.block_1(x))) # <- leverage the benefits of operator fusion